In [ ]:

%load_ext autoreload
%autoreload 2



In [1]:
import os; os.chdir(f'{os.environ["MT_ROOT"]}/src/')
import torch as th
from torch import nn
import torchvision

import constants as ct
from models.i3d import I3D
from models.ae_i3d import AEI3D
from models.vae_i3d import VAEI3D
from models.tadn import TimeAlignedDenseNet
from models.helpers import count_parameters

In [5]:
class A:
    x = None
    
    def __init__(self):
        self.x = nn.ConstantPad3d(1, 0)
class B(nn.Module):
    x = None
    def __init__(self):
        super(B, self).__init__()
        self.x = nn.ConstantPad3d(1, 0)

In [6]:
a = A()
b = B()
a.x, b.x

(ConstantPad3d(padding=(1, 1, 1, 1, 1, 1), value=0), None)

In [7]:
s = nn.Softmax(dim=-1)
x = th.randn((2, 4, 10))
a = s(x).mean(dim=1)
b = s(x.mean(dim=1))
a.shape, b.shape, th.argmax(a), a, th.argmax(b), b

(torch.Size([2, 10]),
 torch.Size([2, 10]),
 tensor(2),
 tensor([[0.0230, 0.0747, 0.1909, 0.0952, 0.1164, 0.0239, 0.1272, 0.1002, 0.1037,
          0.1447],
         [0.0631, 0.1012, 0.1456, 0.0847, 0.0969, 0.0620, 0.0877, 0.1319, 0.1101,
          0.1168]]),
 tensor(2),
 tensor([[0.0304, 0.0685, 0.2404, 0.0789, 0.0904, 0.0251, 0.1346, 0.0874, 0.0902,
          0.1541],
         [0.0601, 0.0942, 0.1411, 0.0627, 0.0904, 0.0638, 0.0866, 0.1537, 0.1155,
          0.1320]]))

In [2]:
i3d = I3D(ct.SMTH_NUM_CLASSES, 0.0)
tadn = TimeAlignedDenseNet(4, 12, 0.0, ct.SMTH_NUM_CLASSES)
aei3d = AEI3D(1024, 0.0, ct.SMTH_NUM_CLASSES)
vaei3d = VAEI3D(1024, 0.0, ct.SMTH_NUM_CLASSES, 'soft')
le_net = torchvision.models.GoogLeNet(num_classes=ct.SMTH_NUM_CLASSES, aux_logits=False)
inception = torchvision.models.inception_v3(num_classes=ct.SMTH_NUM_CLASSES)
res_net = torchvision.models.resnet18(num_classes=ct.SMTH_NUM_CLASSES)
print(f'LeNet:\t\t{count_parameters(le_net)}')
print(f'Inception:\t{count_parameters(inception)}')
print(f'ResNet18:\t{count_parameters(res_net)}')
print(f'I3D:\t\t{count_parameters(i3d)}')
print(f'TADN:\t\t{count_parameters(tadn)}')
print(f'AEI3D:\t\t{count_parameters(aei3d)}')
print(f'VAEI3D:\t\t{count_parameters(vaei3d)}')

LeNet:		5630654
Inception:	24427804
ResNet18:	11191902
I3D:		6598718
TADN:		32611024
AEI3D:		14358178
VAEI3D:		15407748
